In [1]:
import sys
import time

import numpy as np
from osgeo import gdal, osr, ogr

<h1>1 - Compara&ccedil;&atilde;o 1: Recurs&atilde;o em cauda e recurs&atilde;o simples</h1>
<h3>1.1 - Recurs&atilde;o simples:</h3>
<p>Uma recursão simples, primeiramente empilha na memória todas as repetições que deverão ser executadas, então vai desempilhando conforme as mesmas são executadas. Este processo de empilhar e desempilhar na memória torna o processo da recursão simples lento se comparado com um loop <i>for</i></p>




In [2]:
def fatorial_simples(num):
    if num <= 1:
        return 1
    else:
        fat = fatorial_simples(num - 1)
        return fat * num    

<h3>1.2 - Recurs&atilde;o em cauda:</h3>
<p>Uma recurs&atilde;o em cauda ao contr&aacute;rio da recurs&atilde;o simples, executa as repeti&ccedil;&otilde;es enquanto empilha elas, tornando o processo mais r&aacute;pido.</p>

In [3]:
def fatorial_cauda(num):
    if num <= 1:
        return 1
    else:
        return num * fatorial_cauda(num - 1)


<h3>1.3 - Vamos compara-las:</h3>

In [4]:
# Aumenta o limite de chamadas recursivas para 10000
# a quantidade padrao de chamadas é 3000
sys.setrecursionlimit(5000)

# Mostra a quantidade de chamas recursivas que o sistema pode fazer
print("A quantidade de chamadas recursivas que podem ser feitas e {}".format(sys.getrecursionlimit()))

somatorio = 0

for i in range(1000):
    ini = int(round(time.time() * 1000000))
    fatSimples = fatorial_simples(4000)
    fim = int(round(time.time() * 1000000))
    somatorio += (fim - ini)

print("Tempo de execucao fatorial simples "+
      "{} microssegundos".format(somatorio/1000))

somatorio = 0

for i in range(1000):
    ini = int(round(time.time() * 1000000))
    fatCauda = fatorial_cauda(4000)
    fim = int(round(time.time() * 1000000))
    somatorio += (fim - ini)

print("Tempo de execucao fatorial cauda "+
      "{} microssegundos".format(somatorio/1000))


A quantidade de chamadas recursivas que podem ser feitas e 5000
Tempo de execucao fatorial simples 7722.787 microssegundos
Tempo de execucao fatorial cauda 6704.581 microssegundos


<h3>1.4 Conclusão:</h3>
<p>Nesta comparação de código, é possível perceber que a recursão em cauda tende a ser mais rápida. Contudo, como o Python não implementa a recursão em cauda (<i>tail recursion</i>), as duas funções estão sendo processadas praticamente da mesma forma. O que pode estar tornando a recursão em cauda mais rápida pode ser a quantidade das linhas da função, visto que a função possui menos linhas comparado com a recursão simples.<br/>
A principal desvantagem de se utilizar recursão em Python é o seu limite pequeno de chamadas. Apesar de poder modificar este limite, não é recomendado, visto que caso seja feito muitas chamadas, a chance de provocarmos um <i>stack overflow</i> (ou <i>estouro de pilha</i>) é bem alta.</p>

<h1>2 - Compara&ccedil;&atilde;o 2: C&aacute;lculo das coordenadas geogr&aacute;ficas</h1>
<h3>2.1 - Descri&ccedil;&atilde;o da fun&ccedil;&atilde;o:</h3>
<p>Uma das operações mais essenciais do programa é o cálculo das coordenadas geográficas de cada pixel da imagem. Isto porque os dados de treinamento utilizados na classificação supervisionada estão sob coordenadas geográficas. O cruzamento das entradas, imagem e os dados de treinamento, resultam no pixel correspondente para cada ponto de treinamento.</p>

<h3>2.2 - Fun&ccedil;&atilde;o Orientada a Objetos</h3>
<p>Neste código podemos observar que um simples loop <i>for</i> satisfaz o procedimento, sendo necessário apenas iterar por cada pixel e fazer os cálculos necessários. Adicionando em listas distintas as coordenadas x e y.</p>

In [5]:
def get_pixels_coordinates_oo(raster, epsg=32722):
        # Pega as referencias espaciasi do raster
        raster_gtrn = raster.GetGeoTransform()
        crs = osr.SpatialReference()
        crs.ImportFromWkt(raster.GetProjectionRef())
        # Pega o codigo epsg referente a projecao 
        crsGeo = osr.SpatialReference()
        crsGeo.ImportFromEPSG(epsg)
        # Cria um "tradutor" de coordenadas
        transform = osr.CoordinateTransformation(crs, crsGeo)
        # Cria uma lista para as coordenadas x e outra para as y
        X = []
        Y = []
        # Cria um indice para os pixeis da imagem
        #index = np.ndindex(int(raster.RasterXSize / 2), int(raster.RasterYSize / 2))
        index = np.ndindex(raster.RasterXSize, raster.RasterYSize)
        for i in index:
            # Pega a linha e coluna referente ao pixelda matriz
            x, y = i
            # Calcula as coordenadas x e y
            x2 = raster_gtrn[1] * x + raster_gtrn[2] * y + raster_gtrn[0]
            y2 = raster_gtrn[4] * x + raster_gtrn[5] * y + raster_gtrn[3]
            # Calcula as coordenadas do centro do pixel
            x2 += (raster_gtrn[1]/2.0)
            y2 += (raster_gtrn[5]/2.0)
            # Traduz as coordenadas para a projecao do codigo epsg
            cx, cy = transform.TransformPoint(x2, y2)[:2]
            # Adiciona os valores nas listas
            X.append((cx))
            Y.append((cy))
        return X,Y

<h3>2.3 - Fun&ccedil;&atilde;o que calcula as coordenadas do Python Funcional</h3>
<p>Para calcular as coordenadas em Python funcional, é utilizado uma fun&ccedil;&atilde;o Decorator e outra fun&ccedil;&atilde;o que chama o decorator. Em seguida veremos cada uma delas separadamente:</p>
<h3>2.3.1 - Fun&ccedil;&atilde;o Decorator:</h3>
<p>Nesta fun&ccedil;&atilde;o podemos observar que h&aacute; a implementa&ccedil;&atilde;o do padr&atilde;o de projeto (Designer Pattern) Decorator, onde o mesmo é uma fun&ccedil;&atilde;o que recebe outra fun&ccedil;&atilde;o, decorando-a ou adicionando uma funcionalidade na fun&ccedil;&atilde;o de entrada. Vale lembrar que o uso de Decorators n&atilde;o &eacute; caracter&iacute;stico de linguagens funcionais puras.  </p>

In [6]:
def coordenadas(func, funcX, funcY, funcCenter):
    def calculo(index):
        x, y = index
        #C alcula as coordenadas x e y
        x2 = funcX(x, y)
        y2 = funcY(x, y)
        # Calcula as coordenadas do centro do pixel
        x2center = funcCenter(x2)
        y2center = funcCenter(y2)
        # Traduz as coordenadas para a projecao do codigo epsg 
        cx, cy = func(x2center, y2center)[:2]
        return cx, cy

    return calculo

<h3> 2.3.2 - Função que chama o "<i>Decorator</i>" :</h3>
<p>A função abaixo, é a função que de fato calcula as coordenadas de cada pixel.<br/>
Nesta função são criadas funções anônimas. Estas mesmas funções serão decoradas para assim ser realizado o cálculo das coordenadas para cada pixel.<br/>
    Nesta função também foi utilizada a função de alta ordem <i>map</i>. Esta função irá iterar pelo conjunto de dados e aplicar a função que foi passada como atributo..</p>

In [7]:
def get_pixels_coordinates_func(raster, epsg=32722):
    # Pega as referencias espaciasi do raster
    raster_gtrn = raster.GetGeoTransform()
    crs = osr.SpatialReference()
    crs.ImportFromWkt(raster.GetProjectionRef())
    # Pega o codigo epsg referente a projecao 
    crsGeo = osr.SpatialReference()
    crsGeo.ImportFromEPSG(epsg)
    # Cria um "tradutor" de coordenadas
    transform = osr.CoordinateTransformation(crs, crsGeo)
    # Cria um indice para os pixeis da imagem
    #index = np.ndindex(int(raster.RasterXSize / 2), int(raster.RasterYSize / 2))
    index = np.ndindex(raster.RasterXSize, raster.RasterYSize)
    func1 = coordenadas(lambda x, y: transform.TransformPoint(x, y),
                            lambda x, y: raster_gtrn[1] * x + raster_gtrn[2] * y + raster_gtrn[0],
                            lambda x, y: raster_gtrn[4] * x + raster_gtrn[5] * y + raster_gtrn[3],
                            lambda x: x + (raster_gtrn[1] / 2.0))
    
    coor = np.array(list(map(func1, index)))
    # cria uma lista apenas com as coordenadas x
    x = coor[:,0]
    # cria uma lista apenas com as coordenadas y
    y = coor[:,1]
    return x,y

In [8]:
path = r"TesteClassificacao.tif"
raster = gdal.Open(path)
ini = time.time()
coordenadasOO = get_pixels_coordinates_oo(raster)
fim = time.time()
print("Tempo de execucao da funcao orientada a objetos {} segundos".format(fim - ini))

ini = time.time()
coordenadasFunc = get_pixels_coordinates_func(raster)
fim = time.time()
print("Tempo de execucao da funcao funcional {} segundos".format(fim - ini))

Tempo de execucao da funcao orientada a objetos 18.373998880386353 segundos
Tempo de execucao da funcao funcional 42.81450295448303 segundos


<h1>3 - Comparação 3: Função reordenação das classes</h1>
<p>Esta função substitui os valores atuais das classes por novos valores definidos pelo usuário.</p>
<h3>3.1 - Função Orientada a objetos</h3>
<p>A função orientada a objetos usa um <i>loop for</i> para iterar pelos valores das classes.</p>

In [9]:
def replace_array_values_oo(array, actualValues, newValues):
        index = len(actualValues)
        for i in range(index):
            array[array == actualValues[i]] = newValues[i]

<h3> 3.2 - Função funcional</h3>
<p>A função do paradigma funcional utiliza uma recursão em cauda.</p>

In [10]:
def replace_array_values_func(array, actualValues, newValues):
    index = len(actualValues)
    if index == 0 :
         return array
    else:
        array[array == actualValues[0]] = newValues[0]
        return replace_array_values_func(array, actualValues[1:], newValues[1:])
    pass

In [11]:
banda = raster.GetRasterBand(1)
matriz = banda.ReadAsArray()

valoresAtuais = [1,2,3,4,5,6,7,8,9]
valoresNovos = [0,1,2,3,4,5,6,7,8]

matrizOO = matriz.copy()
matrizFunc = matriz.copy()

ini  = time.time()
replace_array_values_oo(matrizOO, valoresAtuais, valoresNovos)
fim = time.time()

print("Tempo de execucao funcao Orientada a Objetos {} segundos".format(fim - ini))

ini = time.time()
arrayNovo = replace_array_values_func(matrizFunc, valoresAtuais ,valoresNovos)
fim = time.time()
print("Tempo de execucao funcao Funcional {} segundos".format(fim - ini))



Tempo de execucao funcao Orientada a Objetos 0.12028121948242188 segundos
Tempo de execucao funcao Funcional 0.07167482376098633 segundos


<h1>4 - Conclusão:</h1>
<table style="font-size: 25px; text-align: center;">
  <tr>
    <th>Características</th>
    <th>Python funcional</th>
    <th>Python Orientado a Objetos</th>
  </tr>
  <tr>
    <td>Escalabilidade</td>
    <td style="color:	#FF0000; font-weight: 700; text-align: center;">-</td>
    <td style="color: #228B22; font-weight: 700; text-align: center;">X</td>
  </tr>
  <tr>
    <td>Facilidade em prototipar</td>
    <td style="color: #228B22; font-weight: 700; text-align: center;">X</td>
    <td style="color:	#FF0000; font-weight: 700; text-align: center;">-</td>
  </tr>
  <tr>
    <td>Imutabilidade dos dados</td>
    <td style="color: #228B22; font-weight: 700; text-align: center;">X</td>
    <td style="color:	#FF0000; font-weight: 700; text-align: center;">-</td>
  </tr>
</table>
